# CS145: Project 1 | Large Dataset, Movies, and Vibes

## Author
Bryan Steven Monge Serrano, bryan101

## Project Overview

---

I want to understand the main trends and types of movies that are popular based off the movies ratings and tags from this large dataset of movie reviews.

The goal is to be able to name some of the top movies betweeen different categories, implement the merging of the various datasets across identifying variables, and be able to see what users/tags/genres are prevalent in the ratings.


---


## Analysis of Dataset

---
There are three csv/data tables related to this movie ratings dataset from Kaggle. The three are movies, ratings, and tags.

movies contains the variables movieId, title, and genres. The only quantitative (but it is functionally qualitative) variable is movieId and the others are qualitative. Important to note is that title also contains the years by which the movies were released at the end of the string value, 'XXXX.' There are movies in the data set from 1891 to 2015 and it contains a total of 27,278 rows/ distinct movies.

ratings contains the variables userId, movieId, rating, and timestamp. These are all quantitative in nature, however, all but rating are qualitative in nature. The movies are rated from a scale of 0.5 to 5. There are 20,000,263 ratings/rows made by 138,493 users.

tags contains the variables userId, movieId, and tag. userId and movieId are qualitative but appear quantitiative, and tag is also qualitative.

Relationship: It is clear that you could merge the data tables into one big table by merging on either userId or movieId depending on the data table with JOINs. These keys would be the primary keys to focus on as they are unique for every row in the three data tables. In particular, movieId is the primary key for movies, and userId and movieId would be the primary keys for ratings and tags. Furthermore, these primary keys are each of the tables foreign keys that tie all of the tables together.

Limitations in the data are that although logically you would be able to merge and add to the dataset by combining with other data tables there could be issues resulting from other tables from other sources using unique identifiers inconsistent with this dataset.

Lastly, this dataset is downloaded from Kaggle but comes from MovieLens, and included over 500 MBs of data.

----

# Guiding List of Questions

**Primary Questions**


1.   What are the "top critically rated movies"? In other words, what movies have high ratings despite being rated by users who are critical, with a average user rating less than 2.5, and who have more than 100 ratings?
2.   What do the users with lowest average user rating watch/rate? In addition, what is the lowest average user rating and is there more than one user with the lowest average user rating?
3.   Are there any "top critically rated movies" from 2010 to 2015? If so, what are they?
4.   What 2010 to 2015 movies do the users with the lowest average user rating watch/rate? In particular, what are their top 10 ranked films as collective group, if any?


**Supplementary Question**



1.   What are the top 10 comedic films with non-alphanumeric tags? Do any of them have more than one non-alphanumeric tag?
2.   What are the top 10 films with a tag related to "Colonial" and "Colonialism"? What are the other tags tied to this film? Lastly, what are the other films that are ranked in the top 10?
3.   Did the average rating of Finding Nemo go up or down in the year of its theatrical re-released when compared to its original release year?

**Queries that highlight trends, anomials, significant features**



1.   (Found in Part 1) The exploration of the series of primary questions highlighted that highly critical users do not rate/watch movies released from 2010 to 2015. The only exception of Inception.
2.   (Found in Part 2) Colonial/Colonialism-tagged films tend to perform worse off on average, but a cult-classic ranked #1, Alien, is an outlier in the data
3.   (Found early on in the trivial queries)
There are large number of movies, 534, that have no ratings





**NOTE:**
I explored this dataset naturally and the questions arose as I found patterns and trends in the data. This is a list of the questions I am answering.


# Setting up the Dataset, Tables, and Visualizations of Tables within Colab



In [ ]:
# Run this cell to authenticate yourself to BigQuery
from google.colab import auth
auth.authenticate_user() # make sure to "Select All" when authenticating
project_id = "cs-145-fall"

In [ ]:
# Initialize BiqQuery client
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

In [ ]:
# Import pandas for dataframe visualization
import pandas as pd



## Data Exploration: Get your feet week: Trivial Queries


*Use at least 2 (non-trivial) sub-queries (5 points).*

Firstly, finding the first 5 movies in the ratings from 2005 sorted alphabetically

In [ ]:
%%bigquery --project $project_id

CREATE TABLE movie_rating.movies_2005 AS (
  SELECT movieId, title # selecting the necessary variables
  FROM movie_rating.movies  # selecting the movies dataframe
  WHERE title LIKE '%2005%' # LIKE looks for the string pattern "2005" in the title variable
  ORDER BY title DESC # sort by name in a descending order
)

Query is running:   0%|          |

""


In [ ]:
# This is cell that implements Pandas to display the tables natively in Colab

# Define query
query = """
SELECT * # selecting all variables
FROM `cs-145-fall.movie_rating.movies_2005` # select the data table
LIMIT 10 # limit the number of rows to 10
"""

# Execute the query and convert to a DataFrame
df = client.query(query).to_dataframe() # call the query and turn to pandas df
df

,movieId,title
0,101307,Árido Movie (2005)
1,33158,xXx: State of the Union (2005)
2,59485,a/k/a Tommy Chong (2005)
3,37653,Zozo (2005)
4,76151,Zizek! (2005)
5,82608,Zerophilia (2005)
6,96522,"Zero Years, The (2005)"
7,81405,Zero Degrees of Separation (2005)
8,40851,Zathura (2005)
9,40962,"Yours, Mine and Ours (2005)"


Secondly, finding movies without any ratings

In [ ]:
%%bigquery --project $project_id

CREATE TABLE movie_rating.movies_without_ratings AS (
  SELECT movieId, title # select the movieId and title variables
  FROM `cs-145-fall.movie_rating.movies` m
  WHERE movieId NOT IN ( # selecting all of the movies with movieIds
    SELECT movieId       # not listed in the ratings table
    FROM `cs-145-fall.movie_rating.ratings` r
    WHERE m.movieId = r.movieId
    )
)

Query is running:   0%|          |

""


In [ ]:
# This is cell that implements Pandas to display the tables natively in Colab

# Define query
query = """
SELECT * # selecting all variables
FROM `cs-145-fall.movie_rating.movies_without_ratings` # select the data table

"""

# Execute the query and convert to a DataFrame
df = client.query(query).to_dataframe() # call the query and turn to pandas df
df

,movieId,title
0,118812,Made in Jamaica (2006)
1,122282,Pride and Prejudice (1980)
2,111318,City Under Siege (Chun sing gai bei) (2010)
3,122155,Weary River (1929)
4,117682,Elvis and Anabelle (2007)
...,...,...
529,117666,Easy to Love (1953)
530,117127,Camino (2008)
531,121279,Little Lord Fauntleroy (1980)
532,89482,Portrait of Maria (María Candelaria (Xochimilc...


Through these two non-trivial queries, we observe some of the films made in 2005 as well as the films that contained no ratings in the ratings table.

## Data Exploration: Get your feet week: Using CTEs

*Using at least two CTEs (Common Table Expressions) and Window Functions*

Using CTEs to find highly critically rated movies. My criteria for highly critically rated films are films that high ratings despite getting a rating from users that on average give low ratings (rating < 2.5). Additionally, I used ROW_NUMBER() and RANK() to label the rows.

In [ ]:
%%bigquery --project $project_id

# Definition: Top Critically Rated Movies are movies that hold a relatively
# high rating despite getting ratings from users that are critical, users
# who on average tend to give low ratings

CREATE TABLE movie_rating.top_critically_rated_movies AS (

WITH user_ratings AS ( # CTE that calculates the average user ratings
  SELECT
    r.userId,
    AVG(r.rating) AS avg_rating
  FROM `cs-145-fall.movie_rating.ratings` r
  GROUP BY r.userId
),

movie_ratings AS ( # CTE that builds on prior CTE to find movies that are rated highly
  SELECT           # despite being rated by critical users (one or more) that give low ratings
    m.movieId,     # on average
    m.title,
    COUNT(r.rating) AS num_ratings,
    AVG(r.rating) AS avg_rating,
  FROM `cs-145-fall.movie_rating.movies` m
  JOIN `cs-145-fall.movie_rating.ratings` r
  ON m.movieId = r.movieId
  WHERE r.userId IN ( # sub-query that filters for films rated by users with
     SELECT userId    # average user rating below 2.5
     FROM user_ratings
     WHERE avg_rating < 2.5
     )
  GROUP BY m.movieId, m.title
)

SELECT           # Query that displays the filtered out results
    title,       # from movie_ratings such that only movies
    avg_rating,  # 100 or more ratings are listed
    num_ratings,
    RANK() OVER (ORDER BY avg_rating DESC) AS rank
FROM movie_ratings
WHERE num_ratings > 100
ORDER BY rank ASC
)

Query is running:   0%|          |

""


In [ ]:
# This is cell that implements Pandas to display the tables natively in Colab

# Define query
query = """
SELECT *
FROM `cs-145-fall.movie_rating.top_critically_rated_movies`
LIMIT 10
"""

# Execute the query and convert to a DataFrame
df = client.query(query).to_dataframe()
df

,title,avg_rating,num_ratings,rank
0,"Usual Suspects, The (1995)",3.386364,440,1
1,Fight Club (1999),3.340085,469,2
2,Pulp Fiction (1994),3.285714,602,3
3,Mulholland Drive (2001),3.275194,129,4
4,Donnie Darko (2001),3.257447,235,5
5,"Departed, The (2006)",3.250000,110,6
6,"Shawshank Redemption, The (1994)",3.248333,600,7
7,Eternal Sunshine of the Spotless Mind (2004),3.247768,224,8
8,Memento (2000),3.239274,303,9
9,American Beauty (1999),3.232104,461,10


I see that most of the top 10 highly rated/ranked films are from the late 90s and early 2000s, which is interesting as that means that the more recent films that were highly rated were likely not rated by these critical users with low average rating scores.

Seeing this made interested in first exploring what movies the users with the lowest average user rating rated/watched as well as what the lowest average user rating was. I was especially curious to see what years the movies were from.

In [ ]:
%%bigquery --project $project_id

# Definition: Top 10 movies for the/a user with the lowest average user rating.
# I used window functions and CTEs to filter through the users and find the/a
# user with the lowest average user rating, and then found their top 10
# rated films

CREATE TABLE movie_rating.top10_movies_lowest_rating_user AS (

WITH user_rankings AS (  # CTE used to calculate user rankings based on ratings
  SELECT
    r.userId,
    ROW_NUMBER() OVER (ORDER BY AVG(r.rating) ASC) AS user_rank # Used ROW_NUMBER
  FROM `cs-145-fall.movie_rating.ratings` r                     # to only choose one user
  GROUP BY r.userId
),
lowest_rated_user AS (   # CTE used to locate the user(s) with the lowest rank
  SELECT userId
  FROM user_rankings
  WHERE user_rank = 1
),
movie_ratings AS (       # CTE used to filter through the movie ratings and find
  SELECT                 # the top 10 rated films for the user with the lowest
    m.movieId,           # average user rating
    m.title,
    r.userId,
    AVG(r.rating) AS avg_rating,
    r.rating AS user_rating,
    RANK() OVER (ORDER BY AVG(r.rating) ASC) AS rank
  FROM `cs-145-fall.movie_rating.movies` AS m
  JOIN `cs-145-fall.movie_rating.ratings` AS r
  ON m.movieId = r.movieId
  WHERE r.userId IN (
    SELECT userId
    FROM lowest_rated_user
    )
  GROUP BY m.movieId, m.title, r.userId, r.rating
)

SELECT
    title,
    avg_rating,
    user_rating,
    rank,
    userId
FROM movie_ratings
WHERE rank <= 10
ORDER BY rank ASC
)

Query is running:   0%|          |

""


In [ ]:
# This is cell that implements Pandas to display the tables natively in Colab

# Define query
query = """
SELECT *
FROM `cs-145-fall.movie_rating.top10_movies_lowest_rating_user`
"""

# Execute the query and convert to a DataFrame
df = client.query(query).to_dataframe()
df

,title,avg_rating,user_rating,rank,userId
0,Deep Impact (1998),0.5,0.5,1,20501
1,Tommy Boy (1995),0.5,0.5,1,20501
2,"Thomas Crown Affair, The (1999)",0.5,0.5,1,20501
3,"Postman, The (Postino, Il) (1994)",0.5,0.5,1,20501
4,Kingpin (1996),0.5,0.5,1,20501
5,Sneakers (1992),0.5,0.5,1,20501
6,Wag the Dog (1997),0.5,0.5,1,20501
7,French Kiss (1995),0.5,0.5,1,20501
8,Splash (1984),0.5,0.5,1,20501
9,"Crying Game, The (1992)",0.5,0.5,1,20501


I saw that for this particular user (user = 24530), none of his rated films had ratings above 0.5. Similarly, as I was building this query and filtered to only look at one user due to the large number of user with the lowest average user rating, I noticed that the average rating for all the other users with the lowest average rating was 0.5, indicating that all users with this average rating rated every film they rated at 0.5 given that the average would be (n * 0.5)/n, where n = number of films.

Additionally, I also noted that this particular user has no films after the late 1990s. This trend was similar enough for the other users as it was rare to find a film from the early 2000s and beyond from these harsh users.

Lastly, all of the films are ranked (rank = 1), further corroborating my previously stated intuition that the lowest rating is 0.5 due the lack of variance in the ranking.

After this, I went back to my first question: whether recent films (from 2010 to 2015) were rated at all by users where highly critical given the criteria for "critical" we set in the "top_critically_rated_movies" query. I used some of the code I built in the previous query as a structure for my filtering and thinking. After this, I checked to see if films from the 2010 to 2015 were found among the films in the "top_critically_rated_movies" table such that they were

In [ ]:
%%bigquery --project $project_id

CREATE TABLE movie_rating.highly_critically_rated_2010_2015_films AS (

WITH movie_ratings AS ( # CTE to find movies that are rated highly
  SELECT           # despite being rated by critical users (one or more) that give low ratings
    m.movieId,
    m.title,
    AVG(r.rating) AS avg_rating,
  FROM `cs-145-fall.movie_rating.movies` m
  JOIN `cs-145-fall.movie_rating.ratings` r
  ON m.movieId = r.movieId
  WHERE (m.title LIKE '%2010%'
  OR m.title LIKE '%2011%'
  OR m.title LIKE '%2012%'
  OR m.title LIKE '%2013%'
  OR m.title LIKE '%2015%')
  GROUP BY m.movieId, m.title
)

SELECT
    MR.title,
    MR.avg_rating,
    CRM.num_ratings,
    CRM.rank AS old_rank,
    RANK() OVER (ORDER BY MR.avg_rating DESC) AS new_rank
FROM movie_ratings AS MR
JOIN `cs-145-fall.movie_rating.top_critically_rated_movies` AS CRM
ON MR.title = CRM.title
ORDER BY new_rank ASC
)

Query is running:   0%|          |

""


In [ ]:
# This is cell that implements Pandas to display the tables natively in Colab

# Define query
query = """
SELECT *
FROM `cs-145-fall.movie_rating.highly_critically_rated_2010_2015_films`
"""

# Execute the query and convert to a DataFrame
df = client.query(query).to_dataframe()
df

,title,avg_rating,num_ratings,old_rank,new_rank
0,Inception (2010),4.156172,120,11,1


After employing all of this filtering, we see that the only movie ever rated by high critically users (according to my original definition of these users) was Incepetion, released in 2010.

Lastly, I combined the previous queries and their code to see if the users with the lowest average user ratings EVER rated a film from 2010 to 2015.

In [ ]:
%%bigquery --project $project_id

CREATE TABLE movie_rating.movies_lowest_rating_users AS ( # ALL USERS WITH LOW RATINGS

WITH user_rankings AS (  # CTE used to calculate user rankings based on ratings
  SELECT
    r.userId,
    RANK() OVER (ORDER BY AVG(r.rating) ASC) AS user_rank # Used RANK
  FROM `cs-145-fall.movie_rating.ratings` r               # to choose all lowest rating users
  GROUP BY r.userId
),
lowest_rated_users AS (   # CTE used to locate the user(s) with the lowest rank
  SELECT userId
  FROM user_rankings
  WHERE user_rank = 1
),
movie_ratings AS (       # CTE used to filter through the movie ratings
  SELECT
    m.movieId,
    m.title,
    r.userId,
    AVG(r.rating) AS avg_rating,
    r.rating AS user_rating,
    RANK() OVER (ORDER BY AVG(r.rating) ASC) AS rank
  FROM `cs-145-fall.movie_rating.movies` AS m
  JOIN `cs-145-fall.movie_rating.ratings` AS r
  ON m.movieId = r.movieId
  WHERE r.userId IN (
    SELECT userId
    FROM lowest_rated_users
    )
  GROUP BY m.movieId, m.title, r.userId, r.rating
)

SELECT
    title,
    avg_rating,
    user_rating,
    rank,
    userId
FROM movie_ratings
ORDER BY rank ASC
)

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project $project_id

CREATE TABLE movie_rating._2010_2015_films_list AS ( # ALL MOVIES from 2010 to 2015

WITH movie_ratings AS ( # CTE that builds on prior CTE to find movies that are rated highly
  SELECT           # despite being rated by critical users (one or more) that give low ratings
    m.movieId,
    m.title,
    AVG(r.rating) AS avg_rating,
    COUNT(r.rating) AS num_ratings,
  FROM `cs-145-fall.movie_rating.movies` m
  JOIN `cs-145-fall.movie_rating.ratings` r
  ON m.movieId = r.movieId
  WHERE (m.title LIKE '%2010%'
  OR m.title LIKE '%2011%'
  OR m.title LIKE '%2012%'
  OR m.title LIKE '%2013%'
  OR m.title LIKE '%2015%')
  GROUP BY m.movieId, m.title
)
SELECT
    MR.title,
    MR.avg_rating,
    MR.num_ratings,
    RANK() OVER (ORDER BY MR.avg_rating DESC) AS rank
FROM movie_ratings AS MR
ORDER BY rank ASC
)

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project $project_id

# Combined result of the above tables

CREATE TABLE movie_rating.movies_lowest_rating_users_2010_2015 AS (
  SELECT L.title, L.avg_rating
  FROM `cs-145-fall.movie_rating._2010_2015_films_list` AS L
  JOIN `cs-145-fall.movie_rating.movies_lowest_rating_users` AS F
  ON L.title = F.title
  GROUP BY L.title, L.avg_rating
  ORDER BY L.avg_rating DESC
)

Query is running:   0%|          |

""


In [ ]:
# Define query
query = """
SELECT *
FROM `cs-145-fall.movie_rating.movies_lowest_rating_users_2010_2015`
"""

# Execute the query and convert to a DataFrame
df = client.query(query).to_dataframe()
df

,title,avg_rating


We see that there are no films that fulfill this criteria :(

**PART 2: Analyzing Tags + GENRE**

Using CTEs to find the comedy films tagged with solely non-alphanumeric characters and ranking them.

In [ ]:
%%bigquery --project $project_id

CREATE TABLE movie_rating.unthought_of_films AS( # A table contain films I wouldn't think of
  WITH tag_counter AS (
    SELECT
      movieId,
      COUNT(*) AS num_tags
    FROM `cs-145-fall.movie_rating.tags`
    WHERE REGEXP_CONTAINS(tag, r'^[^A-Za-z0-9]+$') # REGEXP_CONTAINS looks for regex expressions
    GROUP BY movieId                               # without alphanumeric tags
  ),
  non_english_films_tags AS ( # CTE containing solely films with non-english characters
    SELECT
        R.userId,
        T.movieId,
        R.tag,
        T.num_tags
    FROM `cs-145-fall.movie_rating.tags` AS R
    RIGHT JOIN tag_counter AS T
      ON T.movieId = R.movieId
    WHERE REGEXP_CONTAINS(tag, r'^[^A-Za-z0-9]+$')
  ),
  non_english_films_ratings AS ( # CTE contain the ratings for non-English tagged films
    SELECT
          T.userId,
          T.movieId,
          T.tag,
          T.num_tags,
          AVG(R.rating) AS avg_ratings
    FROM `cs-145-fall.movie_rating.ratings` AS R
    RIGHT JOIN non_english_films_tags AS T     # RIGHT JOIN to ensure that all tagged movies are in the table
    ON T.movieId = R.movieId
    GROUP BY T.userId, T.movieId, T.tag, T.num_tags
  ),
  ranked_films AS ( # CTE containing all of the ranked comedy films that have non-alphanumeric tags
  SELECT
          M.title,
          M.genres,
          R.tag,
          R.num_tags,
          R.avg_ratings,
          RANK() OVER (ORDER BY avg_ratings DESC) AS rank # Ranking the films by avg_ratings and genre
  FROM non_english_films_ratings AS R
  JOIN `cs-145-fall.movie_rating.movies` AS M
  ON R.movieId = M.movieId
  WHERE M.genres LIKE '%Comedy%'
  GROUP BY M.title, M.genres, R.tag, R.num_tags, R.avg_ratings
  ORDER BY rank ASC
  )
  SELECT *           # Making a query with only rank <= 10; the top 10
  FROM ranked_films
  WHERE rank <= 10
  ORDER BY rank ASC
)

Query is running:   0%|          |

""


In [ ]:
# Define query
query = """
SELECT *
FROM `cs-145-fall.movie_rating.unthought_of_films`
"""

# Execute the query and convert to a DataFrame
df = client.query(query).to_dataframe()
df

,title,genres,tag,num_tags,avg_ratings,rank
0,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War,?,1,4.175837,1
1,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,æš´åŠ›,2,4.174231,2
2,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,ä½Žä¿—å° è¯´,2,4.174231,2
3,Snatch (2000),Comedy|Crime|Thriller,==================,1,4.042147,4
4,Forrest Gump (1994),Comedy|Drama|Romance|War,ç» å…¸,1,4.029000,5
5,"Girl Who Leapt Through Time, The (Toki o kaker...",Animation|Comedy|Drama|Romance|Sci-Fi,========================,1,3.924745,6
6,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,é˜®ä¸€é¸£,1,3.921240,7
7,Finding Nemo (2003),Adventure|Animation|Children|Comedy,=========================,1,3.886228,8
8,Groundhog Day (1993),Comedy|Fantasy|Romance,,1,3.867623,9
9,Secondhand Lions (2003),Children|Comedy|Drama,==============================,1,3.746922,10


These are the highest-ranked comedy films with malformed tags or tags that are not in alphanumeric characters. From this, I can see that there are a variety of genres that one film can fall under which I haven't seen before in my prior queries. Additionally, Pulp Fiction stands out as a comedic film that is highly rated with a malformed or non-alphanumeric tag given that it appears to have two. After exploring the data and adding a additional CTE, I can confirm that Pulp Fiction is the only movie with multiple tags (num_tags = 2) under these conditions.

After this, I began checking to see how highly "Colonial/Colonization" films were ranked (rank <= 10). I removed and kept the "rank <= 10" filter in various table iterations to compare and see what films were in the resultant table depending on the ranking.

In [ ]:
%%bigquery --project $project_id

CREATE TABLE movie_rating.colonial_films AS(
  WITH tag_descriptor AS ( # CTE sorting tag information
    SELECT
      movieId,
      COUNT(*) AS num_tags
    FROM `cs-145-fall.movie_rating.tags`
    WHERE tag LIKE '%Colonialism%' OR tag LIKE '%Colonial%'
    GROUP BY movieId
  ),
  colonial_tagged AS ( # CTE containing solely films with colonial tags
    SELECT
        R.userId,
        T.movieId,
        R.tag,
        T.num_tags
    FROM `cs-145-fall.movie_rating.tags` AS R
    LEFT JOIN tag_descriptor AS T
    ON T.movieId = R.movieId
  ),
  movies_ratings_CTE AS ( # CTE containing the ratings for the films
    SELECT
          T.userId,
          T.movieId,
          T.tag,
          T.num_tags,
          AVG(R.rating) AS avg_ratings
    FROM `cs-145-fall.movie_rating.ratings` AS R
    LEFT JOIN colonial_tagged AS T  # Ensuring all tagged movies are in the table
    ON T.movieId = R.movieId
    GROUP BY T.userId, T.movieId, T.tag, T.num_tags
  ),
  ranked_films AS ( # CTE containing all ranked films that have the specified tag
    SELECT         # and all of their other associated tags
          M.title,
          M.genres,
          R.tag,
          R.num_tags,
          R.avg_ratings,
          RANK() OVER (ORDER BY R.avg_ratings DESC) AS rank # Ranking the films by avg_ratings
    FROM movies_ratings_CTE AS R
    JOIN `cs-145-fall.movie_rating.movies` AS M
    ON R.movieId = M.movieId
    GROUP BY M.title, M.genres, R.tag, R.num_tags, R.avg_ratings
  )
  SELECT * # SELECTING the top 10 ranked films
  FROM ranked_films
  WHERE rank <= 10
  ORDER BY rank ASC
)

Query is running:   0%|          |

""


In [ ]:
# Define query
query = """
SELECT *
FROM `cs-145-fall.movie_rating.colonial_films`
"""

# Execute the query and convert to a DataFrame
df = client.query(query).to_dataframe()
df

,title,genres,tag,num_tags,avg_ratings,rank
0,Aliens (1986),Action|Adventure|Horror|Sci-Fi,Katso Sanna!,1,4.005642,1
1,Aliens (1986),Action|Adventure|Horror|Sci-Fi,overrated,1,4.005642,1
2,Aliens (1986),Action|Adventure|Horror|Sci-Fi,space horror,1,4.005642,1
3,Aliens (1986),Action|Adventure|Horror|Sci-Fi,Colonial marines,1,4.005642,1
4,Aliens (1986),Action|Adventure|Horror|Sci-Fi,menacing,1,4.005642,1
...,...,...,...,...,...,...
107,Aliens (1986),Action|Adventure|Horror|Sci-Fi,In The Nucleus,1,4.005642,1
108,Aliens (1986),Action|Adventure|Horror|Sci-Fi,Paul Reiser,1,4.005642,1
109,Aliens (1986),Action|Adventure|Horror|Sci-Fi,SF,1,4.005642,1
110,Aliens (1986),Action|Adventure|Horror|Sci-Fi,Alien,1,4.005642,1


From here, we can see that after exploring the table the only movie that is highly ranked in the rankings that references "Colonial" or "Colonialism" in any of its tags is Alien. We also see all of the other tags and the tag counter and see that the use of words related to "Colonial/Colonialism" is rare for this film.

If you were to remove the rank <= 10 filter, you see that the other films are "Proposition, The" at rank = 113, "Choke" at rank = 144, and "Rebel, The" at rank = 179. From here, we see the ratity of the label and how movies with the tag related to "Colonial/Colonialism" lower ranks on average. Additionally, the tag also seems not to be tied to one genre as a variety of genres are link to the tag from comedy to drama to romance.

After this, I was curious as to whether my favorite film, Finding Nemo, had higher user ratings in 2003, the year of it's release, or in 2012, the year of the re-release in theatres.

In [ ]:
%%bigquery --project $project_id

CREATE TABLE movie_rating.finding_nemo_ratings_2003_2012 AS( # A table contain films I wouldn't think of
  WITH ratings_2003 AS ( # CTE containing the ratings rated in 2003
    SELECT
        R.movieId,
        AVG(R.rating) AS avg_rating_2003,
        EXTRACT(YEAR FROM TIMESTAMP_SECONDS(R.timestamp)) AS year
    FROM `cs-145-fall.movie_rating.ratings` R
    WHERE EXTRACT(YEAR FROM TIMESTAMP_SECONDS(R.timestamp)) = 2003
    GROUP BY R.movieId, year
  ),
  ratings_2012 AS ( # CTE containing the ratings rated in 2012
    SELECT
        R.movieId,
        AVG(R.rating) AS avg_rating_2012,
        EXTRACT(YEAR FROM TIMESTAMP_SECONDS(R.timestamp)) AS year
    FROM `cs-145-fall.movie_rating.ratings` R
    WHERE EXTRACT(YEAR FROM TIMESTAMP_SECONDS(timestamp)) = 2012
    GROUP BY  R.movieId, year
  ),
  combined_ratings AS ( # CTE combining the data from the two CTEs
    SELECT
        COALESCE(r2003.movieId, r2012.movieId) AS movieId, # COALESECE keeps one movieID
        r2003.avg_rating_2003,
        r2012.avg_rating_2012
    FROM ratings_2003 r2003
    FULL OUTER JOIN ratings_2012 r2012 # OUTER JOIN KEEPS ALL ROWS from both
    ON r2003.movieId = r2012.movieId
    ),
  nemo_ratings AS ( # CTE containing the ratings for Nemo
    SELECT
        M.title,
        M.genres,
        R.avg_rating_2003,
        R.avg_rating_2012
    FROM combined_ratings R
    JOIN `cs-145-fall.movie_rating.movies` M
    ON R.movieId = M.movieId
    WHERE M.title LIKE '%Finding Nemo%'
    GROUP BY M.title, M.genres, R.avg_rating_2003, R.avg_rating_2012
  )
  SELECT *           # Making a query that outputs the ratings
  FROM nemo_ratings
)

Query is running:   0%|          |

""


In [ ]:
# Define query
query = """
SELECT *
FROM `cs-145-fall.movie_rating.finding_nemo_ratings_2003_2012`
"""

# Execute the query and convert to a DataFrame
df = client.query(query).to_dataframe()
df

,title,genres,avg_rating_2003,avg_rating_2012
0,Finding Nemo (2003),Adventure|Animation|Children|Comedy,4.15299,3.857903


From here, I can see that the average rating of the Finding Nemo movie did go down when it was re-released when compared to its original release year.

## Conclusion

**Summary**

I learned that there is a lot of subjectivity in regards to the tagging and ratings of movies and saw trends that I didn't expect to see. For instance, what truly shocked me was that highly critical users had not rated many films from the recent era the dataset was published in (around 2016). This to me indicated that these critical users leaned towards watching cult classics and rating those. However, what was amusing was that the only film that was watched/rated by these users was Inception given the pandemic we all lived in from 2020 to 2023.

Additionally, I was curious about leveraging the tags given that I never thought of associating ratings with tags. However, I found interesting results in regard to the colonial-ly tagged films, which is that there is a highly popular movie, Alien, that has this association as of recent given that the recent film in the franchise is set in the 1800s and is between a Comanche woman, a tribe in the Southwest of the US and Northern Mexico, and the Alien and is called "Prey."

Lastly, I made a personal exploration of the ratings of Finding Nemo and was saddened to see a downward trend when comparing the average rating at year of release when compared to around 10 years later.

**Limitations and Discussion**

Given these findings, I was wondering what the trends, tags, and rating would be in a data set with more recent data given the proliferation of rating services, i.e. Letterbox, and the ease at which you can rate something from your phone. I would like to see if this new data would lead to large shifts in the averages of movies with an already high number of ratings in this dataset and if more recent films would have more or a proportional number of ratings relative to their release year due to the ease at which you can rate things now (given that you don't have to wait to logon to your PC at home to make a rating).

Additionally, as such, the main limitation for this dataset would be that the time frame of the data is outdated given that it lacks information for 2016 to 2024.

**Future Exploration**

I would use more time to combine this data set with other data sets that are comparable to compare the average rating of films given the data from this data set and the other (new) datasets. I would also want to explore if the prevelance of the colonial/colonialism label increased given the increased representation of Native actors in films with films like "Killers of the Flower Moon." Additionally, I would be particularly interested in combining this data with a dataset that lists the age ranges of users to get a better sense of who rates films in what way.